In [21]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl import drawing 
import openpyxl

In [22]:
df = pd.read_excel("../input/horaires.xlsx", "input")
wb = load_workbook(filename="../templates/schedule_template.xlsx")

In [23]:
df["sem"] = df.semaine.apply(lambda x: "a" if "A" in x else "b")

In [24]:
template = wb["template"]

In [25]:
# get all tags in sheet
tags = {}
for row in template.iter_rows():
    for cell in row:
        if cell.value:
            if "{" in cell.value:
                tags[cell.value] = {
                    "value" : "", # free all tags content
                    "col" : cell.column,
                    "row": cell.row
                }

In [26]:
print(tags)

{'{classe}': {'value': '', 'col': 4, 'row': 3}, '{j1p0ac}': {'value': '', 'col': 4, 'row': 8}, '{j1p0bc}': {'value': '', 'col': 5, 'row': 8}, '{j2p0ac}': {'value': '', 'col': 6, 'row': 8}, '{j2p0bc}': {'value': '', 'col': 7, 'row': 8}, '{j3p0ac}': {'value': '', 'col': 8, 'row': 8}, '{j3p0bc}': {'value': '', 'col': 9, 'row': 8}, '{j4p0ac}': {'value': '', 'col': 10, 'row': 8}, '{j4p0bc}': {'value': '', 'col': 11, 'row': 8}, '{j5p0ac}': {'value': '', 'col': 12, 'row': 8}, '{j5p0bc}': {'value': '', 'col': 13, 'row': 8}, '{j1p0ap}': {'value': '', 'col': 4, 'row': 9}, '{j1p0bp}': {'value': '', 'col': 5, 'row': 9}, '{j2p0ap}': {'value': '', 'col': 6, 'row': 9}, '{j2p0bp}': {'value': '', 'col': 7, 'row': 9}, '{j3p0ap}': {'value': '', 'col': 8, 'row': 9}, '{j3p0bp}': {'value': '', 'col': 9, 'row': 9}, '{j4p0ap}': {'value': '', 'col': 10, 'row': 9}, '{j4p0bp}': {'value': '', 'col': 11, 'row': 9}, '{j5p0ap}': {'value': '', 'col': 12, 'row': 9}, '{j5p0bp}': {'value': '', 'col': 13, 'row': 9}, '{j1

In [27]:
classes = df.classe.unique()

In [28]:
df

,classe,#day,day,plage,start,end,semaine,cours,pro,/,code horaire,sem
0,1A,1,Lundi,P1,8h45,10h15,AB,Mathématiques,donya.bensaad,0.04,1A,a
1,1A,1,Lundi,P2,10h30,12h,AB,Français,marine.panneels,0.04,1A,a
2,1A,1,Lundi,P3,13h15,14h45,AB,Éducation physique,lisa.rouard,0.04,1A,a
3,1A,1,Lundi,P4,15h,16h30,AB,Technologie,marie-laure.transon,-1.07,1A,a
4,1A,2,Mardi,P1,8h45,10h15,AB,Français,marine.panneels,0.04,1A,a
...,...,...,...,...,...,...,...,...,...,...,...,...
564,6C,5,Vendredi,P2,10h30,12h,AB,Activité complémentaire Mathématiques,nicolas.goossens,-1.13,6C-S61-ACMATH,a
565,6C,5,Vendredi,P2,10h30,12h,AB,Activité complémentaire Mathématiques,nicolas.goossens,-1.13,6C-S62-ACMATH,a
566,6C,5,Vendredi,P2,10h30,12h,AB,Activité complémentaire Sciences,francois.vanliefferinge,-1.13,6C-S61-ACSV,a
567,6C,5,Vendredi,P2,10h30,12h,AB,Activité complémentaire Sciences,francois.vanliefferinge,-1.13,6C-S62-ACSV,a


In [29]:
# Save result
wb.save("../output/schedule_classe.xlsx")

In [30]:
for cl in classes :
    # get all tags from template
    tags = {}
    for row in template.iter_rows():
        for cell in row:
            if cell.value:
                if "{" in cell.value:
                    tags[cell.value] = {
                        "value" : "", # free all tags content
                        "col" : cell.column,
                        "row": cell.row
                    }

    # get the data
    classe = df[df.classe == cl].copy()
    
    # copy template
    sheet = wb.copy_worksheet(wb["template"])
    sheet.title = cl

    # add logo
    img = drawing.image.Image('../templates/Logo Ecole .jpg')
    scale = 0.1
    img.width = img.width * scale
    img.height = img.height * scale
    sheet.add_image(img, 'A1')

    # set title
    tags["{classe}"]["value"] = cl

    # Populate the tags
    for i, r in classe.iterrows():
        
        plage = ""
        if r["plage"] in ["P00", "P01"]:
            plage = f"({r.start} - {r.end})"
        if r["plage"] == "P00":
            r["plage"] = "P0"
        elif r["plage"] == "P01":
            r["plage"] = "P1"
            
        t = f"{{j{r['#day']}{r.plage.lower()}{r['sem']}"

        tags[t+"c}"]["value"] = r['cours'].upper()
        tags[t+"p}"]["value"] = r["pro"]
        tags[t+"l}"]["value"] = r["/"]
        tags[t+"o}"]["value"] = plage
        

    # populate the tags in the new sheet
    for t in tags:
        sheet.cell(row=tags[t]['row'], column=tags[t]['col']).value = tags[t]['value']

    # Merge cells if needed
    for i, r in classe.iterrows():

        plage = ""
        if r["plage"] in ["P00", "P01"]:
            plage = f"({r.start} - {r.end})"
        if r["plage"] == "P00":
            r["plage"] = "P0"
        elif r["plage"] == "P01":
            r["plage"] = "P1"
        
        t = f"{{j{r['#day']}{r.plage.lower()}{r['sem']}"
        
        # if AB course, get the corresponding b tag for merging
        if r.semaine == 'AB':
            tb = f"{{j{r['#day']}{r.plage.lower()}b"
            for j in ['c', 'p', 'o', 'l']:
                sheet.merge_cells(
                    start_row=tags[t+j+"}"]['row'],
                    start_column=tags[t+j+"}"]['col'],
                    end_row=tags[tb+j+"}"]['row'],
                    end_column=tags[tb+j+"}"]['col']
                )  

In [31]:
# Save result
wb.save("../output/schedule_classe.xlsx")